# Extração e seleção de características

As características que são extraídas dos dados dependem de sua natureza. Os dados EMG são sinais elétricos coletados dentro de um período de tempo, portanto são dados no domínio do tempo. As características deste domínio são extraídas dele. Também é possível transformar os dados EMG para o domínio da frequência e extrair características neste domínio. Existem diversas características que podem ser extraídas de cada domínio, entretando nem todas elas serão relevantes. Cada problema se beneficia de características ou combinações delas. Portanto, é preciso que haja uma seleção de características para encontrar a combinação de características que trará melhor resultado na posterior classificação dos dados.

## Reutilizando os passos anteriores

É necessário carregar os dados pré-processados, para dar início à extração de características. No jupyter notebook podemos utilizar o namespace completo de outro notebook:

In [1]:
%run preprocessing.ipynb

(20000, 2)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 1
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 2
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 3
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 4
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 5
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 6
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 7
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em EMG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) \\$
    $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn([x_i - threshold][x_{i+1} - threshold]) \\$
    $ sgn(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $
    $Sugestão: threshold = 0.4$

### Domínio da frequência

1. Auto Regressive (AR)

    > $ - \sum_{j=1}^{\rho}\alpha_j x_{j-1} + w_n $

2. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

3. Mean Frequency (FMN)

    > $ \sum_{j=1}^{M}f_j PSD_j \Big{ / } \sum_{j=1}^{M}PSD_j $

4. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{i=1}^{M}|w|_i $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Desafio 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

### Extraindo características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características VAR & RMS (domínio do tempo) e FDM & MMDF (domínio da frequência).

# Fórmulas ofertadas pelo Professor

In [2]:
from math import prod

# funções auxiliares:

def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def f_j(j, SampleRate, M):
    return (j * SampleRate) / (2 * M)



# funções de extração de características:

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def WL(x):
    return np.sum(np.abs(np.diff(x)), axis = -1)

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2



# WAMP (domínio do tempo)

### WAMP ( Amplitude de Willison): É o contador para cada mudança na amplitude do sinal exceder o threshold . Pode ser calculado pela somatória de 1 até n - 1, da amostra na posição i menos a amostra na posição i + 1,se amostra na posiçao i for maior que o treeshold definido.

In [3]:
def WAMP(x, limiar):
    return np.sum( np.abs(np.diff(x)) > limiar, axis= -1)


# VAR (domínio do tempo)

### VAR (Variância do EMG): É o quão longe as amostras estão do valor médio. Pode ser calculado pela somatória de 1 até n, de amostras x na posição j elevada ao quadrado. Com esse somatório multiplicando por 1 sobre n - 1 múmero de amostras.

# RMS (domínio do tempo)

### RMS (Raiz Quadrada da Média dos Quadrados): É a raiz quadrada da média aritmética dos quadrados dos valores. Pode ser calculada pela raiz quadrada de 1 sobre o número de amostras n, multiplicando pela somatória de 1 até n, de módulo de x na posição j elevado ao quadrado.

# WL (domínio do tempo)

### WL ( Comprimento da Forma de Onda): É o comprimento acumulativo da forma de onda ao longo de um período de tempo. Pode ser calculado pela somatória de 1 até número o de amostras n menos 1, do modulo da diferença da amostra x na posição i + 1 pela amostra x na posição i.

In [4]:
def WL(x):
    return np.sum(np.abs(np.diff(x)), axis = -1)


# Zero Crossing (domínio do tempo)

### ZC (Zero Crossing): É o número de vezes que o sinal do EMG cruza o zero. Pode ser calculado pela somatória de 1 até o número de amostras n, de sgn(x), onde sgn(x) é uma condicional,onde só será verdadeira quando x na posição i multiplicado por x na posição i + 1 é menor ou igual a zero.

In [5]:
#funções de extração de características:

def ZC_Add(data, th):
    
    somatorio = 0
    resultado = 0
    tamanho = len(data)
    
    for i in range(tamanho - 1):
        resultado1 = (data[i] * data[i+1])
        resultado2 = np.abs((data[i] - data[i+1]))
        
        if(resultado1 < 0) and (resultado2 >=  th):
            somatorio += 1
    return somatorio

def ZC(data, th):
    
    x,y,z = data.shape[:3]
    somatorio_final = []
    for i in range(x):
        somatorio_fx = []
        for j in range(y):
            somatorio_fy = []
            for k in range(z):
                somatorio_fz = ZC_Add(data[i][j][k], th)
                
                somatorio_fy.append(somatorio_fz)
            
            somatorio_fx.append(somatorio_fy)
        
        somatorio_final.append(somatorio_fx)
        
    return np.array(somatorio_final)

# PSD (domínio da frequência)

### PSD (Densidade Espectral da Potência): É calculada pelo módulo da frequência na posição j, elevado ao quadrado.

# FMD (domínio da frequência)

### FMD (Frequência Mediana): É a frequência mediana do espectro de frequência. Pode ser calculada pela multiplicaçã de 1 sobre 2 pela somatória de 1 até o número de amostras m, de PSD na posição j.

# FMN (domínio da frequência)

### FMN (Frequência Média): É a frequência média do espectro de frequência. Pode ser calculada pela somatória de 1 até o número de amostras m, da frequência na posição j multiplicada pela PSD na posição j, dividido pela somatória de 1 até o número de amostras m, de PSD na posição j.

In [6]:
# funções auxiliares:

def f_j(j, SampleRate, M):
    return (j * SampleRate) / (2 * M)

#funções de extração de característica:

def FMN_Add(data):
    M = len(data)
    somatorio = 0
    denominador = np.sum(PSD(data), axis=-1)
    
    for j in range(M):
        somatorio += (f_j(j, 200, M) * PSD(data[j])) / denominador
    return somatorio


def FMN(data):
    x,y,z = data.shape[:3]
    somatorio_final = []
    
    for i in range(x):
        somatorio_fx = []
        for j in range(y):
            somatorio_fy = []
            for k in range(z):
                somatorio_fz = FMN_Add(data[i][j][k])
                
                somatorio_fy.append(somatorio_fz)
            
            somatorio_fx.append(somatorio_fy)
        
        somatorio_final.append(somatorio_fx)
        
    return np.array(somatorio_final)

# MMDF (domínio da frequência)

### MMDF (Mediana da Frequência Modificada):É a frequência onde o espectro se divide em duas regiões de iguais amplitudes. É calculada pela multiplicação de 1 sobre 2, pela somatória de 1 até m da amplitude do espectro na posição j.

# MMNF (domínio da frequência)

### MMNF (Média da frequência modificada): é a soma do produto do espectro de amplitude pela frequência, dividido pela soma total das amplitudes do espectro. Pode ser calculada como a somatória de 1 até j, da multiplicação da frequência na posição j,pela amplitude do espectro na posição j, dividido pela somatória de 1 até j, das amplitudes do espectro na posição j.

In [7]:
# funções auxiliares:

def A_j(w):
    return np.abs(w)

#funções de extração de características:

def MMNF_Add(data):
    
    M = len(data)
    somatorio = 0
    denominador = np.sum(A_j(data), axis=-1)
    
    for j in range(M):
        somatorio += ( f_j(j, 200, M) * A_j(data[j])) / denominador
    return somatorio


def MMNF(data):
   
    x,y,z = data.shape[:3]
    somatorio_final = []
    
    for i in range(x):
        somatorio_fx = []
        for j in range(y):
            somatorio_fy = []
            for k in range(z):
                somatorio_fz = MMNF_Add(data[i][j][k])
                
                somatorio_fy.append(somatorio_fz)
            
            somatorio_fx.append(somatorio_fy)
        
        somatorio_final.append(somatorio_fx)
        
    return np.array(somatorio_final)

**Desafio 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características EMG além das apresentadas, citando as fontes de tais características.


## Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

## Implementação do vetor

## Cálculo do PontoMax foi criado no estudo do threshold, para fazer média do ponto mais alto com o a mediana, e posteriormente também o utilizamos para fazer a média com o ponto mais alto com o a média.

In [8]:
PontoMax = np.max(chunks_time)
PontoMax

0.001966231296292373

## A mediana foi necessário no cálculo do WAMP, e também foi estudado a possibilidadede ser utilizada no cálculo do Threshold (MeuLimiar)

In [9]:
Mediana = np.median(chunks_time)
Mediana


2.4288825703437275e-06

## O MeuLimiar seria o threshold, feito pela combinação do ponto mais alto com a mediana.

In [10]:
MeuLimiar = (PontoMax + Mediana) / 2
MeuLimiar

0.0009843300894313584

## O cálculo da média foi feito para o estudo do threshold, na combinação com o ponto mais alto.

In [11]:
Media = np.mean(chunks_time)
Media

-1.3098920532612888e-08

## O MeuLimiarNovo seria o threshold, feito pela combinação do ponto mais alto com a média.

In [12]:
MeuLimiarNovo = (PontoMax + Media) / 2
MeuLimiarNovo

0.0009831090986859204

## Nessa etapa são colocadas cada característica numa mesma lista.

In [13]:
final_data = list()

final_data.append(WAMP(chunks_time, Mediana))
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(WL(chunks_time))
final_data.append(ZC(chunks_time, 0))

final_data.append(fmd(chunks_freq))
final_data.append(FMN(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(MMNF(chunks_freq))


final = np.array(final_data)
final.shape

(9, 60, 2, 41)

## Redimensionalisando o vetor de 4 dimensẽs para 2 dimensões, onde a primeira dimensão irá multiplicar com a terceira e se tornará a segunda dimensão do vetor final, e a segunda dimensão irá multiplicar com a quarta dimensão se tornando a primeira dimensão do vetor final.

É necessário que seja reordenado as dimensões do vetor de características, pois cada característica (de cada canal), deve corresponder à última dimensão do vetor. Por fim, as outras dimensões são concatenadas para o número de amostras.

In [14]:
data = final.transpose(1, 3, 2, 0)
X = data.reshape(data.shape[0]*data.shape[1], data.shape[2]*data.shape[3])
X.shape

(2460, 18)

## Seleção de características

Nesta etapa, são selecionadas as características que mais afetam positivamente no resultado final da classificação. Vamos estudar os métodos de seleção de características nesta [página do projeto sklearn](https://scikit-learn.org/stable/modules/feature_selection.html).

**Desafio 3**: mostrar o resultado para os dados de trabalho, para os seguintes métodos se leção de características:
- VarianceThreshold
- Univariate feature selection
    - escolha o que mais for "interessante": `SelectKBest`, `SelectPercentile` e `GenericUnivariateSelect`
- Recursive feature elimination

# Variance Threshold

In [23]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold = (0.1))
X_Variance = sel.fit_transform(X)

In [24]:
X_Variance.shape

(2460, 8)

# SelectKBest

In [27]:
# criação dos rótulos
# 1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2,...,3,...

y = [[str(i)] * int(X.shape[0] / 10) for i in range(10)]
y = np.array(y).flatten()
print('Shape dos rótulos:', y.shape)

Shape dos rótulos: (2460,)


In [28]:
from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(chi2, k = 10).fit_transform(X, y)


In [29]:
X_new.shape

(2460, 10)